In [1]:
%run common.ipynb

Tensorflow version: 1.14.0
Keras version: 2.2.4
Numpy version: 1.16.5


Using TensorFlow backend.


In [2]:
from deeplift.layers import NonlinearMxtsMode
import deeplift.conversion.kerasapi_conversion as kc
from collections import OrderedDict
import deeplift
from deeplift.util import compile_func
import sys, os

In [3]:
def percent(background, motif_scores):
    p = 0
    for motif_score in motif_scores:
        idx, scores = motif_score
        mean, std = background[idx]
        score = np.mean(scores)
        
        if score > mean - std and score < mean + std:
            p += 1
    return p/len(motif_scores)

In [4]:
def run_explanations(all_runs):
    output = []

    for keras_model_weights, keras_model_json, testing_data_path, method in all_runs:
        data = load_data(method, testing_data_path)

        model = kc.convert_model_from_saved_files(
                    h5_file=keras_model_weights,
                    json_file=keras_model_json,
                    nonlinear_mxts_mode=NonlinearMxtsMode.DeepLIFT_GenomicsDefault)
        score_func = model.get_target_contribs_func(find_scores_layer_idx=0, target_layer_idx=-2)

        background = OrderedDict([('A', 0.3), ('C', 0.2), ('G', 0.2), ('T', 0.3)])

        all_scores = np.zeros((3, data.onehot.shape[0], data.onehot.shape[1]))
        for task_idx in [0,1,2]:
            scores = np.array(score_func(
                        task_idx=task_idx,
                        input_data_list=[data.onehot],
                        input_references_list=[
                         np.array([background['A'],
                                   background['C'],
                                   background['G'],
                                   background['T']])[None,None,:]],
                        batch_size=200,
                        progress_update=None))
            scores = np.sum(scores, axis=2)
            all_scores[task_idx] = scores
        all_scores = np.average(all_scores, axis=0)

        mask = np.zeros(all_scores.shape)
        background = []
        motif_scores = {motif : [] for motif in all_motifs}

        for idx in range(all_scores.shape[0]):
            for embedding in data.embeddings[idx]:
                motif_scores[embedding.what.getDescription()].append(
                    (idx, all_scores[idx, embedding.startPos:embedding.startPos+len(embedding.what)])
                )
                mask[idx, embedding.startPos:embedding.startPos+len(embedding.what)] = 1

            background.append(
                (np.mean(all_scores[idx, np.where(mask[idx] == 0)]), np.std(all_scores[idx, np.where(mask[idx] == 0)]))
                )

        output.append('Background mean: {:.3}, Background std: {:.3}'.format(np.mean(background), np.std(background)))

        for motif in all_motifs:
            if len(motif_scores[motif]) == 0:
                continue
            output.append('Motif: ' + motif)
            output.append('Percent 1 std: {:.2%}'.format(percent(background, motif_scores[motif])))
    return output

## Percent of motifs fall in background ('and' model)

In [11]:
all_and_runs = []
for i in range(len(all_motifs)):
    for j in range(i + 1, len(all_motifs)):
        m1, m2 = all_motifs[i], all_motifs[j]
        all_and_runs.append(('../model/model_and_' + m1 + '_' + m2 + '.h5', 
             '../model/model_and_' + m1 + '_' + m2 + '.json', 
             '../data/testing_' + m1 + '_' + m2 + '.npy', 
             check_and(m1, m2)))
and_results = run_explanations(all_and_runs)

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the preceding linear layer is 2 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 7 the preceding linear layer is 6 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
For layer 10 the preceding linear layer is 9 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the prece

In [12]:
for line in and_results:
    print(line)

Background mean: 0.00147, Background std: 0.051
Motif: SRF_disc1
Percent 1 std: 0.51%
Motif: AP1_disc1
Percent 1 std: 1.31%
Background mean: 0.00194, Background std: 0.0416
Motif: SRF_disc1
Percent 1 std: 0.52%
Motif: GATA_disc1
Percent 1 std: 2.12%
Background mean: -0.00382, Background std: 0.0548
Motif: SRF_disc1
Percent 1 std: 1.14%
Motif: TAL1_known1
Percent 1 std: 1.16%
Background mean: -0.00022, Background std: 0.0503
Motif: SRF_disc1
Percent 1 std: 0.66%
Motif: IRF_known1
Percent 1 std: 0.78%
Background mean: 0.00401, Background std: 0.0486
Motif: AP1_disc1
Percent 1 std: 24.94%
Motif: GATA_disc1
Percent 1 std: 30.29%
Background mean: 0.00416, Background std: 0.0404
Motif: AP1_disc1
Percent 1 std: 2.62%
Motif: TAL1_known1
Percent 1 std: 1.07%
Background mean: -0.000129, Background std: 0.0417
Motif: AP1_disc1
Percent 1 std: 6.02%
Motif: IRF_known1
Percent 1 std: 0.27%
Background mean: -0.000256, Background std: 0.0845
Motif: GATA_disc1
Percent 1 std: 7.64%
Motif: TAL1_known1
Per

## Percent of motifs fall in background ('or' model)

In [13]:
all_or_runs = []
for i in range(len(all_motifs)):
    for j in range(i + 1, len(all_motifs)):
        m1, m2 = all_motifs[i], all_motifs[j]
        all_or_runs.append(('../model/model_and_' + m1 + '_' + m2 + '.h5', 
             '../model/model_and_' + m1 + '_' + m2 + '.json', 
             '../data/testing_' + m1 + '_' + m2 + '.npy', 
             check_or(m1, m2)))
or_results = run_explanations(all_or_runs)

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the preceding linear layer is 2 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 7 the preceding linear layer is 6 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
For layer 10 the preceding linear layer is 9 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the prece

In [14]:
for line in or_results:
    print(line)

Background mean: 0.00147, Background std: 0.051
Motif: SRF_disc1
Percent 1 std: 0.51%
Motif: AP1_disc1
Percent 1 std: 1.31%
Background mean: 0.00194, Background std: 0.0416
Motif: SRF_disc1
Percent 1 std: 0.52%
Motif: GATA_disc1
Percent 1 std: 2.12%
Background mean: -0.00382, Background std: 0.0548
Motif: SRF_disc1
Percent 1 std: 1.14%
Motif: TAL1_known1
Percent 1 std: 1.16%
Background mean: -0.00022, Background std: 0.0503
Motif: SRF_disc1
Percent 1 std: 0.66%
Motif: IRF_known1
Percent 1 std: 0.78%
Background mean: 0.00401, Background std: 0.0486
Motif: AP1_disc1
Percent 1 std: 24.94%
Motif: GATA_disc1
Percent 1 std: 30.29%
Background mean: 0.00416, Background std: 0.0404
Motif: AP1_disc1
Percent 1 std: 2.62%
Motif: TAL1_known1
Percent 1 std: 1.07%
Background mean: -0.000129, Background std: 0.0417
Motif: AP1_disc1
Percent 1 std: 6.02%
Motif: IRF_known1
Percent 1 std: 0.27%
Background mean: -0.000256, Background std: 0.0845
Motif: GATA_disc1
Percent 1 std: 7.64%
Motif: TAL1_known1
Per

## Percent of motifs fall in background ('single motif' model)

In [17]:
all_single_runs = []
for i in all_motifs:
    all_single_runs.append(('../model/model_single_' + i + '.h5', 
         '../model/model_single_' + i + '.json', 
         '../data/testing_' + i + '.npy', 
         check(i)))

single_results = run_explanations(all_single_runs)

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the preceding linear layer is 2 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 7 the preceding linear layer is 6 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
For layer 10 the preceding linear layer is 9 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 1 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 3 the prece

In [18]:
for line in single_results:
    print(line)

Background mean: 0.0138, Background std: 0.0492
Motif: SRF_disc1
Percent 1 std: 0.93%
Background mean: 0.00909, Background std: 0.0311
Motif: AP1_disc1
Percent 1 std: 1.88%
Background mean: 0.0098, Background std: 0.0232
Motif: GATA_disc1
Percent 1 std: 1.62%
Background mean: 0.00868, Background std: 0.0263
Motif: TAL1_known1
Percent 1 std: 0.96%
Background mean: 0.0114, Background std: 0.0288
Motif: IRF_known1
Percent 1 std: 0.47%
